## Import Classes from Libraries

In [41]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

## Read Tangled Prompt and Spec from org-mode export

In [42]:
format_instructions = open("formatter-prompt.txt", "r")
description = open("description.txt", "r")

In [43]:
model = ChatOpenAI(model="gpt-4-turbo-preview")
spec_prompt =  ChatPromptTemplate.from_messages([
  ("system", format_instructions.read()),
  ("user", "Convert the following data flow description to an interaction list: {description}")
])
parser = StrOutputParser()

## Invoke the LLM Chat Model

In [44]:
spec_chain = spec_prompt | model | parser

In [45]:
spec =  spec_chain.invoke({"description": description.read()})

In [46]:
print(spec)

- user sends diagram description (plain text) to orchestrator
- orchestrator sends prompt (plain text) to formatter LLM
- formatter LLM sends spec to orchestrator
- orchestrator sends spec to diagrammer LLM
- diagrammer LLM sends diagram source code to orchestrator
- orchestrator sends diagram source code to diagramming tool
- diagramming tool sends diagram image (PNG) to orchestrator
- orchestrator sends diagram image (PNG) to user


## Diagram Generation


In [47]:
diagram_instructions = open("diagrammer-prompt.txt", "r")
diagram_prompt = ChatPromptTemplate.from_messages([
  ("system", diagram_instructions.read()),
  ("user", "Diagram the following data flow: {spec}")
])

In [48]:
diagram_chain = diagram_prompt | model | parser

In [49]:
diagram_source = diagram_chain.invoke({"spec": spec})

In [50]:
print(diagram_source)

```dot
digraph G {
    graph [bgcolor=white fontname="Roboto Mono" label="Generated by OpenAI's model, version 2023-04"];
    node [style=filled colorscheme=paired12 fontname="Roboto Mono"];
    edge [colorscheme=paired12 fontname="Roboto Mono"];

    user [shape=oval label="user" color=1];
    orchestrator [shape=box label="orchestrator" color=2];
    formatterLLM [shape=box label="formatter LLM" color=3];
    diagrammerLLM [shape=box label="diagrammer LLM" color=4];
    diagrammingtool [shape=box label="diagramming tool" color=5];

    user -> orchestrator [label="diagram description\n(plain text)" color=1 fontcolor=1];
    orchestrator -> formatterLLM [label="prompt\n(plain text)" color=2 fontcolor=2];
    formatterLLM -> orchestrator [label="spec" color=3 fontcolor=3];
    orchestrator -> diagrammerLLM [label="spec" color=4 fontcolor=4];
    diagrammerLLM -> orchestrator [label="diagram source code" color=5 fontcolor=5];
    orchestrator -> diagrammingtool [label="diagram source co